** Checking the Allows **

In [2]:
import requests

robots = 'https://craigslist.org/robots.txt'
response_robots  = requests.get(robots)
print(response_robots.text)

User-agent: *
Disallow: /reply
Disallow: /fb/
Disallow: /suggest
Disallow: /flag
Disallow: /mf
Disallow: /eaf




** Creating a list of the listings **

Craigslist is split up according to location. Since I wanted vehicles from all locations, I needed to record all of the URLs 'stems' to scrape.

In [3]:
from bs4 import BeautifulSoup
import re
import numpy as np
sites = 'https://www.craigslist.org/about/sites#US'
response_sites = requests.get(sites)
page_sites = response_sites.text
print(response_sites.status_code)
soup_sites = BeautifulSoup(page_sites, "lxml")
# Getting all the URLs for across the US
US_sites = soup_sites.find(class_ = 'colmask')
url_sites = [] 
for link in US_sites.find_all('a', href = True): 
    url_sites.append(link['href'])
# Creating a list of all car truck searches across the US
cars_urls = []
for urls in url_sites:
    cars_urls.append(urls + 'search/cta?s=')

cars_urls_2 = [x for x in cars_urls if "craigslist.org/" in x]
cars_urls_2

200


['https://auburn.craigslist.org/search/cta?s=',
 'https://bham.craigslist.org/search/cta?s=',
 'https://dothan.craigslist.org/search/cta?s=',
 'https://shoals.craigslist.org/search/cta?s=',
 'https://gadsden.craigslist.org/search/cta?s=',
 'https://huntsville.craigslist.org/search/cta?s=',
 'https://mobile.craigslist.org/search/cta?s=',
 'https://montgomery.craigslist.org/search/cta?s=',
 'https://tuscaloosa.craigslist.org/search/cta?s=',
 'https://anchorage.craigslist.org/search/cta?s=',
 'https://fairbanks.craigslist.org/search/cta?s=',
 'https://kenai.craigslist.org/search/cta?s=',
 'https://juneau.craigslist.org/search/cta?s=',
 'https://flagstaff.craigslist.org/search/cta?s=',
 'https://mohave.craigslist.org/search/cta?s=',
 'https://phoenix.craigslist.org/search/cta?s=',
 'https://prescott.craigslist.org/search/cta?s=',
 'https://showlow.craigslist.org/search/cta?s=',
 'https://sierravista.craigslist.org/search/cta?s=',
 'https://tucson.craigslist.org/search/cta?s=',
 'https://yu

I loop through all of the location URL stems, finding the number of pages of listings and putting the URLs into a list to loop through.

In [4]:
list_listings = []
for urls in cars_urls_2:
    listings = urls
    #print(listings)
    response_listings = requests.get(listings)
    page_listings = response_listings.text
    soup_listings = BeautifulSoup(page_listings, "lxml")
    # Find number of pages
    script = [x.find_all('span') for x in soup_listings.find_all('span')]
    total_cars = re.compile('"totalcount">[0-9]+').findall(str(script))
    try:
        total_cars_2 = int(total_cars[1].replace('"totalcount">',""))
    except(IndexError):
        continue
    cars_per_page = re.compile('"rangeTo">[0-9]+').findall(str(script))
    try:
        cars_per_page_2 = int(cars_per_page[1].replace('"rangeTo">',""))
    except(IndexError):
        continue
    pages = list(np.arange(cars_per_page_2,total_cars_2,cars_per_page_2)) 
    first_page = ''
    pages.insert(0,first_page)
    #print(pages)
    for page in pages:
        listings = urls
        listings += str(page)
        #print(listings)
        response_listings = requests.get(listings)
        page_listings = response_listings.text
        soup_listings = BeautifulSoup(page_listings, "lxml")
        for link in soup_listings.find_all('a', href = True): 
            try:
                list_listings.append(link['href'])
            except(IndexError):
                continue

list_listings_2a = [x for x in list_listings if "craigslist.org/" in x]


There picks up some non-listing URLs and so I exclude those. I also exclude one-by-one the listing pages that seem to bug. I then select just the unique listings.

In [5]:
len(list_listings_2a)

509758

In [ ]:
print(len(list_listings_2a))
list_listings_2 = [x for x in list_listings_2a if not "www.craigslist.org/" in x]
list_listings_2 = [x for x in list_listings_2a if not "https://bakersfield.craigslist.org/ctd/d/we-got-the-best-financing-for/6473778313.html" in x] 
list_listings_2 = [x for x in list_listings_2a if not "//www.craigslist.org/about/terms.of.use.en" in x] 
list_listings_2 = [x for x in list_listings_2a if not "https://sacramento.craigslist.org/cto/d/2009-toyota-rav4-limited/6477747669.html" in x] 
print(len(list_listings_2))
list_listings_unique = set(list_listings_2)
print(len(list_listings_unique))

** Extracting info from one listing **

For each listing, I turn the page into a BeautifulSoup object. I then extract the title, price and attributes. 

In the longer text, I search for words associated with wheelchair accessible vehicles as that I'm interested in identifying these vehicles specifically and mark a dummy as equal to 1 if a relevant term is mentioned.

I save the data to a CSV every 100 listings, and print the progress every 1000 listings.

Much of the for loop is error handling, for example, in case the connection breaks or I exceed the site's limit for hit rate or if the specific tag I'm searching for doesn't exist for that listing.

In [ ]:
import copy
import pandas as pd
cars_list = []
features = {}
for test in list(list_listings_unique):
    #print("URL", test)
    # Getting raw HTML
    try:
        response_test = requests.get(test)
    except(ConnectionError, MaxRetryError):
        time.sleep(320)
        response_test = requests.get(test)        
    page_test = response_test.text
    soup_test = BeautifulSoup(page_test, "lxml")
    # Getting the attributes
    try:
        title = soup_test.find(id = 'titletextonly').text
    except(AttributeError):
        continue
    try:
        price = soup_test.find(class_ = 'price').text
    except(AttributeError):
        continue
    attributes = [x.find_all('span') for x in soup_test.find_all(class_ = "attrgroup")]
    # Putting list of lists into one list
    attributes_list = [item for sublist in attributes for item in sublist]
    #print(attributes_list)
    # Get specific vehicle details
    attribute_vehicle = [re.search(r'(<b>)(.*)(</b>)', str(x).replace(' ','_')) for x in attributes_list]
    #print(attribute_vehicle)
    attribute_vehicle_2 = []
    for a in range(0,len(attribute_vehicle)):
        try:
            attribute_vehicle_2.append(attribute_vehicle[a].group(2))
        except(AttributeError):
            continue
    # Get attribute categories
    attribute_categories = [re.search(r'(<span>)(.*)(:)', str(x).replace(' ','_')) for x in attributes_list]
    attribute_categories_2 = []
    for a in range(0,len(attribute_categories)):
        try:
            attribute_categories_2.append(attribute_categories[a].group(2))
        except:
            attribute_categories_2.append('title')  
        # Get posting text
    try:
        posting = soup_test.find(id = 'postingbody').text
    except(AttributeError):
        posting = None
    handicap = re.search('handicap', posting)   
    wheelchair = re.search('wheelchair', posting)
    #ramp = re.search('ramp', posting)
    #accessible = re.search('accessible', posting)
    features = dict(zip(attribute_categories_2,attribute_vehicle_2))
    if handicap != None or wheelchair != None:
        features['disabled'] = 'Yes'
    else:
        features['disabled'] = 'No'   
    features['URL'] = test
    features['Price'] = price
    cars_list.append(copy.deepcopy(features))
    if len(cars_list) % 100 == 0:
        n = len(cars_list)
        l = len(cars_list)
        data = pd.DataFrame(cars_list[n-100:n])     
        data.to_csv('Craigs_list_{0}.csv'.format(l))
    if len(cars_list) % 1000 == 0:
        print(len(cars_list))

I test a listing at random.

In [ ]:
print(test)
print(len(cars_list))
list(list_listings_unique)[35910]